## 0. Введение

**В данном блокноте будет отображен процесс выполнения проекта №1 по курсу "Анализ данных в индустрии 2023"**



*   Исполнитель: Боровой Алексей Андреевич
*   Группа: Вольнослушатели

**Задание:**

Для того чтобы отслеживать, как выглядит путь клиента от знакомства с продуктом до его получения, и какие в нем есть проблемные места, аналитики часто строят воронки. Этот инструмент позволяет понять, сколько клиентов "теряется" на каждом из этапов процесса и сколько в итоге совершает целевое действие.

Перед вами схема данных проекта Тинькофф квест. Постройте воронку, которая показывала бы в динамике по неделям и месяцам, сколько клиентов посещает наш сайт, и какая доля из них регистрируется на нем \ подает заявку на игру \ по факту приходит на нее. Количество клиентов и конверсии необходимо изобразить на одном графике.

Какие выводы можно сделать исходя из построенной воронки, какие точки роста продукта вы видите?

**Как выполнять задание:**



1.   Подключиться к базе данных с помощью Python в JupyterNotebook \ Google Colab. 
2.   Собрать датасет с необходимыми полями.
3. Посчитать конверсии и сделать визуализацию на Python.
4. Прикрепить файл с выполненным заданием.

**Конверсия** — это процент пользователей от общего количества посетителей сайта, которые выполнили целевое действие (в нашем случае - пришли на игру):


\begin{align}
Целевая\ конверсия = \frac{Количество\ клиентов\ пришло\ на\ игру}{Количество\ пользователей\ зашло\ на\ сайт}*100\%
\end{align}

**Промежуточная конверсия** — действия, которые посетитель совершает, пока добирается до целевого действия.

Например, конверсия этапа "Посещение сайта -> Регистрация на сайте" будет выглядеть следующим образом:


\begin{align}
Промежуточная\ конверсия= \frac{Количество\ пользователей\ зарегистрировалось\ на\ сайте} {Количество\ пользователей\ зашло\ на\ сайт}*100\%
\end{align}

# 1. Импорт необходимых библиотек

In [ ]:
import psycopg2
import pandas as pd
from plotly import graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# 2. Подключение к базе данных

Создаем подключение к учебной базе данных.

In [ ]:
conn = psycopg2.connect(
host="158.160.52.106",
port=5432,
database="postgres",
user="student",
password="JvLda93aA",
)
cur = conn.cursor()

# 3. Формирование датасета

В отдельную переменную поместим текст SQL запроса (предварительно сформированный в DBeaver). Для удобства группировок по месяцам и нелелям сформируем 2 SQL запроса и 2 Датасета.

In [ ]:
sql_query_months = '''
select
	(to_char(visit_dttm, 'YYYY ') || (to_char(visit_dttm, 'Month'))) as year_month,
	date_trunc('month', visit_dttm)::date as first_day_of_month,
	count(distinct c.client_rk) as clients_visited_site,
	count(distinct a.account_rk) as clients_registered_on_site,
	count(distinct ap.account_rk) as clients_applicated_game,
	count(distinct ap.account_rk) filter (
where
	game_flg = 1) as accounts_in_game
from
		msu_analytics.client c
left join msu_analytics.account a
		using(client_rk)
left join msu_analytics.application ap
		using(account_rk)
left join msu_analytics.game g
		using(game_rk)
group by
	1,
	2
order by
	2
'''
sql_query_weeks = '''
select
	date_trunc('week', visit_dttm)::date as first_day_of_week,
	count(distinct c.client_rk) as clients_visited_site,
	count(distinct a.account_rk) as clients_registered_on_site,
	count(distinct ap.account_rk) as clients_applicated_game,
	count(distinct ap.account_rk) filter (
where
	game_flg = 1) as accounts_in_game
from
		msu_analytics.client c
left join msu_analytics.account a
		using(client_rk)
left join msu_analytics.application ap
		using(account_rk)
left join msu_analytics.game g
		using(game_rk)
group by
	1
order by
	1
'''

На основе SQL запроса создаем два датафрейма (с группировкой по неделям и месяцам).

In [ ]:
df_months = pd.read_sql(sql_query_months, conn)
df_weeks = pd.read_sql(sql_query_weeks, conn)

Проверяем, как подгрузились данные.

In [ ]:
df_months.head(10)

,year_month,first_day_of_month,clients_visited_site,clients_registered_on_site,clients_applicated_game,accounts_in_game
0,2022 September,2022-09-01,778,190,51,39
1,2022 October,2022-10-01,843,217,65,56
2,2022 November,2022-11-01,786,200,56,50
3,2022 December,2022-12-01,805,196,57,33
4,2023 January,2023-01-01,852,213,64,20
5,2023 February,2023-02-01,32,8,0,0


In [ ]:
df_weeks.head(10)

,first_day_of_week,clients_visited_site,clients_registered_on_site,clients_applicated_game,accounts_in_game
0,2022-08-29,111,32,7,5
1,2022-09-05,184,31,6,5
2,2022-09-12,167,43,16,8
3,2022-09-19,181,45,12,11
4,2022-09-26,195,59,17,17
5,2022-10-03,204,58,18,15
6,2022-10-10,190,44,13,13
7,2022-10-17,171,41,12,10
8,2022-10-24,189,48,15,11
9,2022-10-31,197,47,16,14


Датасеты сформировались, теперь можно закрыть соединение с БД.

In [ ]:
cur.close()
conn.close()

Добавим к нашим датасетам столбцы с конверсиями (целевой и промежуточными).

In [ ]:
df_months['conv_register'] = (df_months['clients_registered_on_site'] / df_months['clients_visited_site'] * 100).round(2)
df_months['conv_application'] = (df_months['clients_applicated_game'] / df_months['clients_visited_site'] * 100).round(2)
df_months['conv_games'] = (df_months['accounts_in_game'] / df_months['clients_visited_site'] * 100).round(2)

In [ ]:
df_weeks['conv_register'] = (df_weeks['clients_registered_on_site'] / df_weeks['clients_visited_site'] * 100).round(2)
df_weeks['conv_application'] = (df_weeks['clients_applicated_game'] / df_weeks['clients_visited_site'] * 100).round(2)
df_weeks['conv_games'] = (df_weeks['accounts_in_game'] / df_weeks['clients_visited_site'] * 100).round(2)

Проверим значения.

In [ ]:
df_months.head(5)

,year_month,first_day_of_month,clients_visited_site,clients_registered_on_site,clients_applicated_game,accounts_in_game,conv_register,conv_application,conv_games
0,2022 September,2022-09-01,778,190,51,39,24.42,6.56,5.01
1,2022 October,2022-10-01,843,217,65,56,25.74,7.71,6.64
2,2022 November,2022-11-01,786,200,56,50,25.45,7.12,6.36
3,2022 December,2022-12-01,805,196,57,33,24.35,7.08,4.10
4,2023 January,2023-01-01,852,213,64,20,25.00,7.51,2.35


In [ ]:
df_weeks.head(5)

,first_day_of_week,clients_visited_site,clients_registered_on_site,clients_applicated_game,accounts_in_game,conv_register,conv_application,conv_games
0,2022-08-29,111,32,7,5,28.83,6.31,4.50
1,2022-09-05,184,31,6,5,16.85,3.26,2.72
2,2022-09-12,167,43,16,8,25.75,9.58,4.79
3,2022-09-19,181,45,12,11,24.86,6.63,6.08
4,2022-09-26,195,59,17,17,30.26,8.72,8.72


Столбцы с конверсиями посчитались, теперь можно визуализировать данные.

# 4. Построение графиков (визуализация)

Построим график в разрезе месяцев. Изобразим конверсии столбчатыми диаграммами на фоне линейного графика изменения количества посетителей сайта.

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_months['year_month'],
        y=df_months['clients_visited_site'],
        mode="lines+markers+text",
        name="Количество посетителей сайта, чел",
        yaxis="y2",
        marker_color = '#629293',
        text=df_months['clients_visited_site'],
        textposition="top center"
    ))
fig.add_trace(go.Bar(x=df_months['year_month'],
                y=df_months['conv_register'],
                name="Конверсия 'посещение сайта -> регистрация на сайте', %",
                marker_color='#94ABF3',
                text=df_months['conv_register']
                ))
fig.add_trace(go.Bar(x=df_months['year_month'],
                y=df_months['conv_application'],
                name="Конверсия 'посещение сайта -> заявка на игру', %",
                marker_color='#9FD2B8',
                text=df_months['conv_application']
                ))
fig.add_trace(go.Bar(x=df_months['year_month'],
                y=df_months['conv_games'],
                name="Конверсия 'посещение сайта -> участие в игре', %",
                marker_color='#BDABF6',
                text=df_months['conv_games']
                ))
fig.update_layout(
    legend=dict(orientation="h"),
    title='Изменение конверсий по месяцам',
    yaxis=dict(
        title=dict(text="Конверсия, %"),
        side="left",
        range=[0, 30]
    ),
        yaxis2=dict(
        title=dict(text="Количество посетителей сайта, чел."),
        side="right",
        range=[0, 1200],
        overlaying="y")
)

fig.show()

Построим сводный график в разрезе недель. В данном случае поступим наоборот: покажем линейные графики изменения конверсий на фоне столбчатой диаграммы, отображающей количество посететелей сайта.

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_weeks['first_day_of_week'],
        y=df_weeks['conv_register'],
        name="Конверсия 'посещение сайта -> регистрация на сайте', %",
        yaxis="y2",
        marker_color = '#107CB6'
    ))

fig.add_trace(
    go.Scatter(
        x=df_weeks['first_day_of_week'],
        y=df_weeks['conv_application'],
        name="Конверсия 'посещение сайта -> заявка на игру', %",
        yaxis="y2",
        marker_color = '#28A310'
    ))

fig.add_trace(
    go.Scatter(
        x=df_weeks['first_day_of_week'],
        y=df_weeks['conv_games'],
        name="Конверсия 'посещение сайта -> участие в игре', %",
        yaxis="y2",
        marker_color = '#8C1E06'
    ))

fig.add_trace(
    go.Bar(
        x=df_weeks['first_day_of_week'],
        y=df_weeks['clients_visited_site'],
        name = 'Количество посетителей сайта, чел.',
        marker_color = '#87CFCD',
        text=df_weeks['clients_visited_site']
    ))
fig.update_layout(
    title='Изменение конверсий по неделям',
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Количество посетителей сайта, чел"),
        side="left",
        range=[0, 250]
    ),
    yaxis2=dict(
        title=dict(text="Конверсия, %"),
        side="right",
        range=[0, 40],
        overlaying="y"
  
    ),
)
fig.show()

В разрезе недель очень интересен график, показывающий цветом изменение конверсии в зависимости от количества посещений сайта. Даже без цифр легко определить в какой неделе конверсия минимальна (столбец самый темный).

In [ ]:
fig = px.bar(df_weeks, x="first_day_of_week", y='clients_visited_site', color="conv_games", title="Clients visited site -> accounts in game", text_auto=True)
fig.layout.coloraxis.colorbar.title = 'Конверсия, %'
fig.update_layout(
    title = 'Изменение конверсии "Посещения сайта -> участие в игре", %',
    xaxis_title = 'Недели',
    yaxis_title = 'Количество посететителей сайта, чел.'
)
fig.show()

# 5. Выводы

На основе анализа графиков изменения конверсий в размере месяцев и недель можно сделать следующие выводы:
1. Февраль 2023 года не дает полную картину конверсии, поскольку имеются данные за неполный месяц.
2. Целевая конверсия в феврале 2023 года равна 0 поскольку пользователи еще не успели подать заявку на игру и посетить игру.
3. Если не учитывать нулевую целевую конверсию февраля 2023 года, наименьшие целевые конверсии за рассматриваемый период получились в декабре 2022 и январе 2023 - 4.1% и 2.35% соответственно (при относительно стабильных промежуточных конверсиях 7.08% и 7.51% соответственно). Хотя просмотры сайта в январе 2023 максимальны, а в декабре 2022 достаточно высоки. Это может быть обусловлено тем, что в перед новым годом люди активно занимаются приобретением подарков родным и близким, а в новогодние праздники люди стараются проводить время с семьей. Так же можно предположить, что в начале января 2023 некоторые организации, работающие по франшизе, закрыты на выходные, получается, что клиент мог посетить игру только во второй половине месяца.
4. Минимальное количество просмотров сайта в первую неделю сентября 2022 можно объяснить началом учебного года у детей и студентов.
5. Считается, что "нормальный" показатель целевой конверсии составляет порядка 4-4,5%. Таким образом период с сентября 2022 по декабрь 2022 года можно считать условно приемлемым по конверсии пользователей. Повысить целевую конверсию можно следующими способами:

*   Повысить привлекательность сайта (внешний вид, реклама на сторонних ресурсах);
*   Сделать регистрацию на сайте максимально простой (регистрация по номеру телефона, синхронизация с google, vk аккаунтами и проч.);
*   Подбирать для игр удобные для клиента даты;
*   Проводить когортный анализ и А/Б тесты.  

